In [4]:
import pandas as pd
df = pd.read_pickle("../data/dataframes/DT/dt_2021.pkl")
df


,PT,AU,AF,TI,SO,DT,DE,ID,AB,C1,...,SC,UT,CN,AI,big_data,IOT,computing_infrastructure,blockchain,robotics,additive_manufacturing
2,J,"Khalil, A; Mbarek, N; Togni, O","Khalil, Ahmad; Mbarek, Nader; Togni, Olivier",A Self-Optimizing QoS-Based Access for IoT Env...,WIRELESS PERSONAL COMMUNICATIONS,Article,Internet of things; QoS; Self-management; IEEE...,INTERNET,"Nowadays, providing Internet of Things (IoT) e...","[Khalil, Ahmad; Mbarek, Nader; Togni, Olivier]...",...,Telecommunications,WOS:000655557200005,"France, France, France",False,False,True,False,False,False,False
3,J,"Alhejazi, MM; Mohammad, RMA","Alhejazi, Manal Mohamed; Mohammad, Rami Mustaf...",Enhancing the blockchain voting process in IoT...,INFORMATION SECURITY JOURNAL,Article,Internet of things (IoT); blockchain; consensu...,INTERNET,Internet of Things (IoT) is expected to improv...,"[Alhejazi, Manal Mohamed] Imam Abdulrahman Bin...",...,Computer Science,WOS:000604000300001,"Saudi Arabia, Saudi Arabia",False,False,True,False,False,False,False
4,C,"Mlika, Z; Cherkaoui, S","Mlika, Zoubeir; Cherkaoui, Soumaya",Competitive Algorithms and Reinforcement Learn...,IEEE INTERNATIONAL CONFERENCE ON COMMUNICATION...,Proceedings Paper,Internet of things; non-orthogonal multiple ac...,POWER ALLOCATION,This paper studies the problem of massive Inte...,"[Mlika, Zoubeir; Cherkaoui, Soumaya] Univ Sher...",...,Telecommunications,WOS:000719386000041,"Canada, Canada",False,False,True,False,False,False,False
5,C,"Benaddi, H; Jouhari, M; Ibrahimi, K; Bensliman...","Benaddi, Hafsa; Jouhari, Mohammed; Ibrahimi, K...",Securing IoT Transactions Against Double-Spend...,2021 IEEE GLOBAL COMMUNICATIONS CONFERENCE (GL...,Proceedings Paper,Bitcoin Blockchain; Internet of Things; Signal...,NaN,With considerable demand for higher throughput...,"[Benaddi, Hafsa; Jouhari, Mohammed; Ibrahimi, ...",...,Computer Science; Engineering; Telecommunications,WOS:000790747203020,"Morocco, Morocco, Morocco, France",False,False,True,False,False,False,False
7,C,"Boucetta, C; Nour, B; Cusin, A; Moungla, H","Boucetta, Cherifa; Nour, Boubakr; Cusin, Alber...",QoS in IoT Networks based on Link Quality Pred...,IEEE INTERNATIONAL CONFERENCE ON COMMUNICATION...,Proceedings Paper,Internet of Things (IoT); Link quality; QoS; TSCH,NaN,The success of the Internet of Things (IoT) de...,"[Boucetta, Cherifa] Univ Reims, CReSTIC EA 380...",...,Telecommunications,WOS:000719386000152,"France, France, France, China, China",False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,J,"Oo, YM; Prateepchaikul, G; Somnuk, K","Oo, Ye Min; Prateepchaikul, Gumpon; Somnuk, Krit",Continuous acid-catalyzed esterification using...,ULTRASONICS SONOCHEMISTRY,Article,3D printing; Hydrodynamic cavitation; Free fat...,NaN,Free fatty acid (FFA) content in FFA-rich mixe...,"[Oo, Ye Min; Prateepchaikul, Gumpon; Somnuk, K...",...,Acoustics; Chemistry,WOS:000629682300001,"Thailand, Thailand, Thailand",False,False,False,False,False,False,True
496,J,"Sakhakarmy, M; Tian, SY; Raymond, L; Xiong, GP...","Sakhakarmy, Manish; Tian, Siyu; Raymond, Lily;...",Printability study of self-supporting graphene...,INTERNATIONAL JOURNAL OF ADVANCED MANUFACTURIN...,Article,Graphene oxide; Nanoclay; Printability; Self-s...,NaN,"Graphene, a two-dimensional (2D) carbon allotr...","[Sakhakarmy, Manish; Tian, Siyu; Raymond, Lily...",...,Automation & Control Systems; Engineering,WOS:000628481300001,"United States, United States, United States, U...",False,False,False,False,False,False,True
497,J,"Lawson, S; Farsad, A; Rezaei, F; Ludlow, D; Ro...","Lawson, Shane; Farsad, Alireza; Rezaei, Fateme...",Direct Ink Writing of Metal Oxide/H-ZSM-5 Cata...,ACS APPLIED MATERIALS & INTERFACES,Article,additive manufacturing; metal oxide-doped zeol...,3D-PRINTED ZSM-5 MONOLITHS; SELECTIVE FORMATIO...,"Previously, 3D printing of porous materials an...","[Lawson, Shane; Farsad, Alireza; Rezaei, Fatem...",...,Science 

In [10]:

df.DE

2      Internet of things; QoS; Self-management; IEEE...
3      Internet of things (IoT); blockchain; consensu...
4      Internet of things; non-orthogonal multiple ac...
5      Bitcoin Blockchain; Internet of Things; Signal...
7      Internet of Things (IoT); Link quality; QoS; TSCH
                             ...                        
495    3D printing; Hydrodynamic cavitation; Free fat...
496    Graphene oxide; Nanoclay; Printability; Self-s...
497    additive manufacturing; metal oxide-doped zeol...
498    carbon short fiber reinforced concrete; multi-...
499    Cartilage regeneration; Tissue engineering; 3D...
Name: DE, Length: 289289, dtype: object

In [1]:
import pandas as pd
df_sdg = pd.read_pickle("../data/dataframes/SDG/sdg_clean.pkl")
df_sdg

,PT,AU,AF,TI,SO,LA,DT,DE,ID,AB,...,TC,Z9,PY,WC,DI,SC,UT,CN,Target,SDG
0,J,"Bacq, S; Hertel, C; Lumpkin, GT","Bacq, Sophie; Hertel, Christina; Lumpkin, G. T.",Communities at the nexus of entrepreneurship a...,JOURNAL OF BUSINESS VENTURING,English,Review,Community; Entrepreneurship; Societal impact; ...,SOCIAL ENTREPRENEURSHIP; ENTERPRISE; RESILIENC...,Although there is wide recognition of the impo...,...,1,1.0,2022,Business,10.1016/j.jbusvent.2022.106231,Business & Economics,WOS:000814384700001,"United States, United States, Switzerland",{8.3},{8}
1,J,"Blanco, C; Raurich, X","Blanco, Cesar; Raurich, Xavier",Agricultural composition and labor productivity,JOURNAL OF DEVELOPMENT ECONOMICS,English,Article,Structural change; Agricultural composition; L...,STRUCTURAL-CHANGE; SIZE DISTRIBUTION; URBANIZA...,Labor productivity differences between develop...,...,0,0.0,2022,Economics,10.1016/j.jdeveco.2022.102934,Business & Economics,WOS:000833960600001,"Paraguay, Spain","{11.a, 8.3}","{8, 11}"
2,J,"Czemiel-Grzybowska, W","Czemiel-Grzybowska, Wioletta",Conceptualization and Mapping of Predictors of...,ENERGIES,English,Article,technology entrepreneurship; innovations; know...,INNOVATION; BUSINESS; IMPACT,This study seeks to address the issue of the d...,...,0,0.0,2022,Energy & Fuels,10.3390/en15186543,Energy & Fuels,WOS:000858305900001,Poland,"{8.3, 3.3}","{8, 3}"
3,J,"Ekstrom, KM; Jonsson, H","Ekstrom, Karin M.; Jonsson, Hakan",Orchestrating retail in small cities,JOURNAL OF RETAILING AND CONSUMER SERVICES,English,Article,Retailing; Small cities; Orchestration; Experi...,MAIN STREET; PERFORMANCE; AUTHENTICITY; KNOWLE...,The structural transformation of retail is cha...,...,0,0.0,2022,Business,10.1016/j.jretconser.2022.103008,Business & Economics,WOS:000793645800007,"Sweden, Sweden",{8.3},{8}
4,J,"Ji, JY; Liu, LP; Wang, PP; Wu, C; Dong, HX","Ji, Jianyue; Liu, Luping; Wang, Pingping; Wu, ...",The Upgrading of Fishery Industrial Structure ...,AGRICULTURE-BASEL,English,Article,China; fishery; the industrial upgrading; infl...,NaN,Aquatic products have made an increasingly imp...,...,0,0.0,2022,Agronomy,10.3390/agriculture12091342,Agriculture,WOS:000857431800001,"China, China, China, China, China","{8.2, 14.4, 8.3}","{8, 14}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1491291,J,"Malafronte, L; Ahrne, L; Innings, F; Jongsma, ...","Malafronte, L.; Ahrne, L.; Innings, F.; Jongsm...",Prediction of regions of coalescence and agglo...,CHEMICAL ENGINEERING RESEARCH & DESIGN,English,Article,Dairy products; Spray drying; Coalescence; Agg...,DRYING PROCESS; FOOD POWDERS; STICKINESS; CFD;...,Spray drying is a well-established method used...,...,15,15.0,2015,"Engineering, Chemical",10.1016/j.cherd.2015.10.011,Engineering,WOS:000367629900066,"Sweden, Sweden, Sweden, Sweden, Sweden",{2.3},{2}
1491292,J,"Boamah, F","Boamah, Festus","Imageries of the contested concepts ""land grab...",GEOFORUM,English,Article,Biofuels investments; Discourses; Contested co...,FABLES; MYTHS,"In Ghana, the contested concepts of ""land grab...",...,40,40.0,2014,Geography,10.1016/j.geoforum.2013.10.009,Geography,WOS:000337776400037,Norway,{2.3},{2}
1491293,J,"Carr, JA; D'Odorico, P; Suweis, S; Seekell, DA","Carr, Joel A.; D'Odorico, Paolo; Suweis, Samir...",What commodities and countries impact inequali...,ENVIRONMENTAL RESEARCH LETTERS,English,Article,food security; inequality footprint; Gini coef...,SECURITY,The global distribution of food production is ...,...,6,6.0,2016,Environmental Sciences; Meteorology & Atmosphe...,10.1088/1748-9326/11/9/095013,Environmental Sciences & Ecology; Meteorology ...,WOS:000385393300009,"United States, United States, United States, I...",{2.3},{2}
1491294,J,"Exner, A; Bartels, LE; Windhaber, M; Fritz, S;...","Exner, Andreas; Bartels, Lara E.; Windhaber, M...",Constructing landscapes of value: Capitalist i...,LAND USE POLICY,English,Article,Biofuels; Marginal land; Unused land; Land gra...,MIOMBO WOODL

In [3]:
sc = df_sdg.SC

In [5]:
set_sc =
for elem in sc:
    print(elem)
    break

Business & Economics


In [22]:
lst_dt = list(df.columns[21:28])
lst_dt

['AI',
 'big_data',
 'IOT',
 'computing_infrastructure',
 'blockchain',
 'robotics',
 'additive_manufacturing']

In [23]:
df1_cols = set(lst_dt + ['UT'])
df2_cols = set(df.columns)
df2_cols = df2_cols-set(lst_dt)

In [24]:
df2_cols

{'AB',
 'AF',
 'AU',
 'C1',
 'CN',
 'DE',
 'DI',
 'DT',
 'EM',
 'ID',
 'NR',
 'PT',
 'PY',
 'RP',
 'SC',
 'SO',
 'TC',
 'TI',
 'UT',
 'WC',
 'Z9'}

In [25]:
df_to_group = df[list(df1_cols)]
df_main = df[list(df2_cols)]
df_main = df_main.drop_duplicates(subset="UT")

In [26]:
def group_dt(my_set):
    if len(my_set) == 2:
        return True
    else:
        return my_set.pop()

In [27]:
df_to_group = df_to_group.groupby('UT', as_index=False).agg(set)
df_to_group

,UT,IOT,AI,computing_infrastructure,additive_manufacturing,big_data,robotics,blockchain
0,WOS:000535804800005,{False},{True},{False},{False},{False},{False},{False}
1,WOS:000536451200004,{False},{False},{False},{False},{True},{False},{False}
2,WOS:000542869600010,{False},{True},{False},{False},{False},{False},{False}
3,WOS:000542869600017,{False},{False},{False},{False},{False},{True},{False}
4,WOS:000543713000001,{False},{False},{False},{False},{False},{True},{False}
...,...,...,...,...,...,...,...,...
231722,WOS:000887932600002,{True},{False},{False},{False},{False},{False},{False}
231723,WOS:000887932900001,{False},{True},{False},{False},{False},{False},{False}
231724,WOS:000887932900002,{False},"{False, True}",{False},{False},"{False, True}",{False},{False}
231725,WOS:000887933300001,{False},{True},{False},{False},{False},{False},{False}


In [28]:
for dt in lst_dt:
    df_to_group[dt] = df_to_group[dt].apply(group_dt)

In [29]:
df_to_group

,UT,IOT,AI,computing_infrastructure,additive_manufacturing,big_data,robotics,blockchain
0,WOS:000535804800005,False,True,False,False,False,False,False
1,WOS:000536451200004,False,False,False,False,True,False,False
2,WOS:000542869600010,False,True,False,False,False,False,False
3,WOS:000542869600017,False,False,False,False,False,True,False
4,WOS:000543713000001,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...
231722,WOS:000887932600002,True,False,False,False,False,False,False
231723,WOS:000887932900001,False,True,False,False,False,False,False
231724,WOS:000887932900002,False,True,False,False,True,False,False
231725,WOS:000887933300001,False,True,False,False,False,False,False


In [30]:
df_final = df_main.merge(df_to_group, on='UT')
df_final

,DT,SC,C1,EM,AB,Z9,PY,DI,CN,AU,...,SO,TI,ID,IOT,AI,computing_infrastructure,additive_manufacturing,big_data,robotics,blockchain
0,Article,Telecommunications,"[Khalil, Ahmad; Mbarek, Nader; Togni, Olivier]...",ahmad.khalil@u-bourgogne.fr; nader.mbarek@u-bo...,"Nowadays, providing Internet of Things (IoT) e...",0,2021,10.1007/s11277-021-08589-8,"France, France, France","Khalil, A; Mbarek, N; Togni, O",...,WIRELESS PERSONAL COMMUNICATIONS,A Self-Optimizing QoS-Based Access for IoT Env...,INTERNET,True,False,False,False,False,False,False
1,Article,Computer Science,"[Alhejazi, Manal Mohamed] Imam Abdulrahman Bin...",rmmohammad@iau.edu.sa,Internet of Things (IoT) is expected to improv...,5,2022,10.1080/19393555.2020.1869356,"Saudi Arabia, Saudi Arabia","Alhejazi, MM; Mohammad, RMA",...,INFORMATION SECURITY JOURNAL,Enhancing the blockchain voting process in IoT...,INTERNET,True,True,False,False,False,False,True
2,Proceedings Paper,Telecommunications,"[Mlika, Zoubeir; Cherkaoui, Soumaya] Univ Sher...",zoubeir.mlika@usherbrooke.ca; soumaya.cherkaou...,This paper studies the problem of massive Inte...,0,2021,10.1109/ICC42927.2021.9500285,"Canada, Canada","Mlika, Z; Cherkaoui, S",...,IEEE INTERNATIONAL CONFERENCE ON COMMUNICATION...,Competitive Algorithms and Reinforcement Learn...,POWER ALLOCATION,True,True,False,False,False,False,False
3,Proceedings Paper,Computer Science; Engineering; Telecommunications,"[Benaddi, Hafsa; Jouhari, Mohammed; Ibrahimi, ...",hafsa.benaddi@uit.ac.ma; jouhari4med@gmail.com...,With considerable demand for higher throughput...,0,2021,10.1109/GLOBECOM46510.2021.9685598,"Morocco, Morocco, Morocco, France","Benaddi, H; Jouhari, M; Ibrahimi, K; Bensliman...",...,2021 IEEE GLOBAL COMMUNICATIONS CONFERENCE (GL...,Securing IoT Transactions Against Double-Spend...,NaN,True,False,False,False,False,False,True
4,Proceedings Paper,Telecommunications,"[Boucetta, Cherifa] Univ Reims, CReSTIC EA 380...",cherifa.boucetta@univ-reims.fr; n.boubakr@bit....,The success of the Internet of Things (IoT) de...,0,2021,10.1109/ICC42927.2021.9500396,"France, France, France, China, China","Boucetta, C; Nour, B; Cusin, A; Moungla, H",...,IEEE INTERNATIONAL CONFERENCE ON COMMUNICATION...,QoS in IoT Networks based on Link Quality Pred...,NaN,True,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231722,Article,Acoustics; Chemistry,"[Oo, Ye Min; Prateepchaikul, Gumpon; Somnuk, K...",krit.s@psu.ac.th,Free fatty acid (FFA) content in FFA-rich mixe...,12,2021,10.1016/j.ultsonch.2020.105419,"Thailand, Thailand, Thailand","Oo, YM; Prateepchaikul, G; Somnuk, K",...,ULTRASONICS SONOCHEMISTRY,Continuous acid-catalyzed esterification using...,NaN,False,False,False,True,False,False,False
231723,Article,Automation & Control Systems; Engineering,"[Sakhakarmy, Manish; Tian, Siyu; Raymond, Lily...",yifeij@unr.edu,"Graphene, a two-dimensional (2D) carbon allotr...",5,2021,10.1007/s00170-021-06870-5,"United States, United States, United States, U...","Sakhakarmy, M; Tian, SY; Raymond, L; Xiong, GP...",...,INTERNATIONAL JOURNAL OF ADVANCED MANUFACTURIN...,Printability study of self-supporting graphene...,NaN,False,False,False,True,False,False,False
231724,Article,Science & Technology - Other Topics; Materials...,"[Lawson, Shane; Farsad, Alireza; Rezaei, Fatem...",rownaghia@mst.edu,"Previously, 3D printing of porous materials an...",6,2021,10.1021/acsami.0c20752,"United States, United States, United States, U...","Lawson, S; Farsad, A; Rezaei, F; Ludlow, D; Ro...",...,ACS APPLIED MATERIALS & INTERFACES,Direct Ink Writing of Metal Oxide/H-ZSM-5 Cata...,3D-PRINTED ZSM-5 MONOLITHS; SELECTIVE FORMATIO...,False,False,False,True,False,False,False
231725,Article,Chemistry; Materials Science; Metallurgy & Met...,"[Lauff, Philipp; Fischer, Oliver] Tech Univ Mu...",massivbau@tum.de; zfp.zfp@ed.tum.de; matthias....,Carbon fiber reinforcement used in concrete ha...,0,2021,10.3390/ma14227005,"German

In [31]:
from utils import filter_for_eu_countries
df_final = filter_for_eu_countries(df_final, extended=False, how="add")
df_final

,DT,SC,C1,EM,AB,Z9,PY,DI,CN,AU,...,TI,ID,IOT,AI,computing_infrastructure,additive_manufacturing,big_data,robotics,blockchain,EU
0,Article,Telecommunications,"[Khalil, Ahmad; Mbarek, Nader; Togni, Olivier]...",ahmad.khalil@u-bourgogne.fr; nader.mbarek@u-bo...,"Nowadays, providing Internet of Things (IoT) e...",0,2021,10.1007/s11277-021-08589-8,"France, France, France","Khalil, A; Mbarek, N; Togni, O",...,A Self-Optimizing QoS-Based Access for IoT Env...,INTERNET,True,False,False,False,False,False,False,True
1,Article,Computer Science,"[Alhejazi, Manal Mohamed] Imam Abdulrahman Bin...",rmmohammad@iau.edu.sa,Internet of Things (IoT) is expected to improv...,5,2022,10.1080/19393555.2020.1869356,"Saudi Arabia, Saudi Arabia","Alhejazi, MM; Mohammad, RMA",...,Enhancing the blockchain voting process in IoT...,INTERNET,True,True,False,False,False,False,True,False
2,Proceedings Paper,Telecommunications,"[Mlika, Zoubeir; Cherkaoui, Soumaya] Univ Sher...",zoubeir.mlika@usherbrooke.ca; soumaya.cherkaou...,This paper studies the problem of massive Inte...,0,2021,10.1109/ICC42927.2021.9500285,"Canada, Canada","Mlika, Z; Cherkaoui, S",...,Competitive Algorithms and Reinforcement Learn...,POWER ALLOCATION,True,True,False,False,False,False,False,False
3,Proceedings Paper,Computer Science; Engineering; Telecommunications,"[Benaddi, Hafsa; Jouhari, Mohammed; Ibrahimi, ...",hafsa.benaddi@uit.ac.ma; jouhari4med@gmail.com...,With considerable demand for higher throughput...,0,2021,10.1109/GLOBECOM46510.2021.9685598,"Morocco, Morocco, Morocco, France","Benaddi, H; Jouhari, M; Ibrahimi, K; Bensliman...",...,Securing IoT Transactions Against Double-Spend...,NaN,True,False,False,False,False,False,True,True
4,Proceedings Paper,Telecommunications,"[Boucetta, Cherifa] Univ Reims, CReSTIC EA 380...",cherifa.boucetta@univ-reims.fr; n.boubakr@bit....,The success of the Internet of Things (IoT) de...,0,2021,10.1109/ICC42927.2021.9500396,"France, France, France, China, China","Boucetta, C; Nour, B; Cusin, A; Moungla, H",...,QoS in IoT Networks based on Link Quality Pred...,NaN,True,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231722,Article,Acoustics; Chemistry,"[Oo, Ye Min; Prateepchaikul, Gumpon; Somnuk, K...",krit.s@psu.ac.th,Free fatty acid (FFA) content in FFA-rich mixe...,12,2021,10.1016/j.ultsonch.2020.105419,"Thailand, Thailand, Thailand","Oo, YM; Prateepchaikul, G; Somnuk, K",...,Continuous acid-catalyzed esterification using...,NaN,False,False,False,True,False,False,False,False
231723,Article,Automation & Control Systems; Engineering,"[Sakhakarmy, Manish; Tian, Siyu; Raymond, Lily...",yifeij@unr.edu,"Graphene, a two-dimensional (2D) carbon allotr...",5,2021,10.1007/s00170-021-06870-5,"United States, United States, United States, U...","Sakhakarmy, M; Tian, SY; Raymond, L; Xiong, GP...",...,Printability study of self-supporting graphene...,NaN,False,False,False,True,False,False,False,False
231724,Article,Science & Technology - Other Topics; Materials...,"[Lawson, Shane; Farsad, Alireza; Rezaei, Fatem...",rownaghia@mst.edu,"Previously, 3D printing of porous materials an...",6,2021,10.1021/acsami.0c20752,"United States, United States, United States, U...","Lawson, S; Farsad, A; Rezaei, F; Ludlow, D; Ro...",...,Direct Ink Writing of Metal Oxide/H-ZSM-5 Cata...,3D-PRINTED ZSM-5 MONOLITHS; SELECTIVE FORMATIO...,False,False,False,True,False,False,False,False
231725,Article,Chemistry; Materials Science; Metallurgy & Met...,"[Lauff, Philipp; Fischer, Oliver] Tech Univ Mu...",massivbau@tum.de; zfp.zfp@ed.tum.de; matthias....,Carbon fiber reinforcement used in concrete ha...,0,2021,10.3390/ma14227005,"Germany, Germany, Germany, Germany, Germany, G...","Lauff, P; Pugacheva, P; Rutzen, M; Weiss, U; F...",...,Evaluation of the Behavior of Carbon Short Fib...,CEMENT-BASED COMPOSITES; ACOUSTIC-EMISSION; FA...,False,False,False,True,False,False,False,True


In [32]:
df_dt = pd.read_pickle("../data/dataframes/DT/all_digital.pkl")
df_dt

,PT,AU,TI,SO,DE,AB,C1,EM,TC,PY,...,UT,CN,AI,big_data,IOT,computing_infrastructure,blockchain,robotics,additive_manufacturing,EU
0,J,"Ondra, V; Sever, IA; Schwingshackl, CW",A method for detection and characterisation of...,MECHANICAL SYSTEMS AND SIGNAL PROCESSING,Non-linear system characterisation; Hilbert tr...,This paper presents a method for detection and...,"[Ondra, V.; Schwingshackl, C. W.] Imperial Col...",v.ondra14@imperial.ac.uk; Ibrahim.Sever@Rolls-...,26,2017,...,WOS:000385209000012,"United Kingdom, United Kingdom, United Kingdom",True,False,False,False,False,False,False,True
1,J,"Liu, K; Kang, G",Multiview convolutional neural networks for lu...,INTERNATIONAL JOURNAL OF IMAGING SYSTEMS AND T...,lung nodule classification; multiview convolut...,To find a better way to screen early lung canc...,"[Liu, Kui; Kang, Guixia] Beijing Univ Posts & ...",liukui_006@126.com,55,2017,...,WOS:000397734700002,"China, China, China",True,False,False,False,False,False,False,False
2,J,"Ali, MS; Balasubramaniam, P; Zhu, QX",Stability of stochastic fuzzy BAM neural netwo...,INTERNATIONAL JOURNAL OF MACHINE LEARNING AND ...,Discrete and distributed time-varying delay; G...,"Among the various fuzzy models, the well-known...","[Ali, M. Syed] Thiruvalluvar Univ, Dept Math, ...",syedgru@gmail.com; zqx22@126.com,39,2017,...,WOS:000393309600022,"India, India, China",True,False,False,False,False,False,False,False
3,J,"Benvidi, A; Abbasi, S; Gharaghani, S; Tezerjan...",Spectrophotometric determination of synthetic ...,FOOD CHEMISTRY,Multicomponent analysis; Artificial neural net...,"Four common food colorants, containing tartraz...","[Benvidi, Ali; Abbasi, Saleheh; Tezerjani, Mar...",abenvidi@yazd.ac.ir,48,2017,...,WOS:000388112500048,,True,False,False,False,False,False,False,False
4,J,"Rajagopal, K; Balakrishnan, SN; Busemeyer, JR",Neural Network-Based Solutions for Stochastic ...,IEEE TRANSACTIONS ON NEURAL NETWORKS AND LEARN...,Adaptive dynamic programming; neural network-b...,"In this paper, an offline approximate dynamic ...","[Rajagopal, Karthikeyan] Gen Motors Co, Global...",krg2d@mst.edu; bala@mst.edu; jbusemey@indiana.edu,12,2017,...,WOS:000395980500005,"United States, United States, United States, I...",True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253123,J,"Wei, FF; Feng, JQ",Real-time ray casting of algebraic B-spline su...,COMPUTERS & GRAPHICS-UK,Piecewise algebraic surface; B-spline; Newton-...,Piecewise algebraic B-spline surfaces (ABS sur...,"[Wei, Feifei; Feng, Jieqing] Zhejiang Univ, St...",weifeifeicad@gmail.com; jqfeng@cad.zju.edu.cn,2,2011,...,WOS:000294034600008,"China, China",False,False,False,True,False,False,False,False
1253124,C,"Hu, Y; Huang, XM; Baker, AH; Tseng, YH; Bryan,...",Improving the Scalability of the Ocean Barotro...,PROCEEDINGS OF SC15: THE INTERNATIONAL CONFERE...,parallel computing; linear solver; ocean modeling,High-resolution climate simulations are increa...,"[Hu, Yong; Huang, Xiaomeng; Yang, Guangwen] Ts...",huyong11@tsinghua.edu.cn; hxm@tsinghua.edu.cn;...,10,2015,...,WOS:000382162500043,"China, China, China, United States, United Sta...",False,False,False,True,False,False,False,False
1253125,J,"Kim, TJ; Sun, X; Yoon, SE",T-ReX: Interactive Global Illumination of Mass...,IEEE TRANSACTIONS ON VISUALIZATION AND COMPUTE...,Massive models; ray tracing; photon mapping; g...,We propose several interactive global illumina...,"[Kim, Tae-Joon; Yoon, Sung-Eui] Korea Adv Inst...",tjkim.kaist@gmail.com; sunxin@microsoft.com; s...,6,2014,...,WOS:000338118700013,"South Korea, South Korea, China",False,False,False,True,False,False,False,False
1253126,J,"Kavouklis, C; Kallinderis, Y",Parallel adaptation of general three-dimension...,JOURNAL OF COMPUTATIONAL PHYSICS,Parallel computing; Hybrid meshes; Mesh adapta...,A new parallel dynamic mesh adaptation and loa...,"[Kavouklis, Christos; Kallinderis, Yannis] Uni...",christos.kavou

In [33]:
cols_dt = df_dt.columns


In [34]:
df_final = df_final[cols_dt]

In [35]:
conc = pd.concat([df_dt, df_final])
conc


,PT,AU,TI,SO,DE,AB,C1,EM,TC,PY,...,UT,CN,AI,big_data,IOT,computing_infrastructure,blockchain,robotics,additive_manufacturing,EU
0,J,"Ondra, V; Sever, IA; Schwingshackl, CW",A method for detection and characterisation of...,MECHANICAL SYSTEMS AND SIGNAL PROCESSING,Non-linear system characterisation; Hilbert tr...,This paper presents a method for detection and...,"[Ondra, V.; Schwingshackl, C. W.] Imperial Col...",v.ondra14@imperial.ac.uk; Ibrahim.Sever@Rolls-...,26,2017,...,WOS:000385209000012,"United Kingdom, United Kingdom, United Kingdom",True,False,False,False,False,False,False,True
1,J,"Liu, K; Kang, G",Multiview convolutional neural networks for lu...,INTERNATIONAL JOURNAL OF IMAGING SYSTEMS AND T...,lung nodule classification; multiview convolut...,To find a better way to screen early lung canc...,"[Liu, Kui; Kang, Guixia] Beijing Univ Posts & ...",liukui_006@126.com,55,2017,...,WOS:000397734700002,"China, China, China",True,False,False,False,False,False,False,False
2,J,"Ali, MS; Balasubramaniam, P; Zhu, QX",Stability of stochastic fuzzy BAM neural netwo...,INTERNATIONAL JOURNAL OF MACHINE LEARNING AND ...,Discrete and distributed time-varying delay; G...,"Among the various fuzzy models, the well-known...","[Ali, M. Syed] Thiruvalluvar Univ, Dept Math, ...",syedgru@gmail.com; zqx22@126.com,39,2017,...,WOS:000393309600022,"India, India, China",True,False,False,False,False,False,False,False
3,J,"Benvidi, A; Abbasi, S; Gharaghani, S; Tezerjan...",Spectrophotometric determination of synthetic ...,FOOD CHEMISTRY,Multicomponent analysis; Artificial neural net...,"Four common food colorants, containing tartraz...","[Benvidi, Ali; Abbasi, Saleheh; Tezerjani, Mar...",abenvidi@yazd.ac.ir,48,2017,...,WOS:000388112500048,,True,False,False,False,False,False,False,False
4,J,"Rajagopal, K; Balakrishnan, SN; Busemeyer, JR",Neural Network-Based Solutions for Stochastic ...,IEEE TRANSACTIONS ON NEURAL NETWORKS AND LEARN...,Adaptive dynamic programming; neural network-b...,"In this paper, an offline approximate dynamic ...","[Rajagopal, Karthikeyan] Gen Motors Co, Global...",krg2d@mst.edu; bala@mst.edu; jbusemey@indiana.edu,12,2017,...,WOS:000395980500005,"United States, United States, United States, I...",True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231722,J,"Oo, YM; Prateepchaikul, G; Somnuk, K",Continuous acid-catalyzed esterification using...,ULTRASONICS SONOCHEMISTRY,3D printing; Hydrodynamic cavitation; Free fat...,Free fatty acid (FFA) content in FFA-rich mixe...,"[Oo, Ye Min; Prateepchaikul, Gumpon; Somnuk, K...",krit.s@psu.ac.th,12,2021,...,WOS:000629682300001,"Thailand, Thailand, Thailand",False,False,False,False,False,False,True,False
231723,J,"Sakhakarmy, M; Tian, SY; Raymond, L; Xiong, GP...",Printability study of self-supporting graphene...,INTERNATIONAL JOURNAL OF ADVANCED MANUFACTURIN...,Graphene oxide; Nanoclay; Printability; Self-s...,"Graphene, a two-dimensional (2D) carbon allotr...","[Sakhakarmy, Manish; Tian, Siyu; Raymond, Lily...",yifeij@unr.edu,4,2021,...,WOS:000628481300001,"United States, United States, United States, U...",False,False,False,False,False,False,True,False
231724,J,"Lawson, S; Farsad, A; Rezaei, F; Ludlow, D; Ro...",Direct Ink Writing of Metal Oxide/H-ZSM-5 Cata...,ACS APPLIED MATERIALS & INTERFACES,additive manufacturing; metal oxide-doped zeol...,"Previously, 3D printing of porous materials an...","[Lawson, Shane; Farsad, Alireza; Rezaei, Fatem...",rownaghia@mst.edu,7,2021,...,WOS:000611066000073,"United States, United States, United States, U...",False,False,False,False,False,False,True,False
231725,J,"Lauff, P; Pugacheva, P; Rutzen, M; Weiss, U; F...",Evaluation of the Behavior of Carbon Short Fib...,MATERIALS,carbon short fiber reinforced concrete; multi-...,Carbon fiber reinforcement used in concrete ha...,"[Lauff, Philipp; Fischer, Oliver] Tech Univ Mu...",massivbau@tum.de; zfp.zfp@ed.tum.de; matthias...

In [36]:
df1_cols = set(lst_dt + ['UT'])
df2_cols = set(conc.columns)-set(lst_dt)


df_to_group = conc[list(df1_cols)]
df_main = conc[list(df2_cols)]
df_main = df_main.drop_duplicates(subset="UT")
df_to_group = df_to_group.groupby('UT', as_index=False).agg(set)
for dt in lst_dt:
    df_to_group[dt] = df_to_group[dt].apply(group_dt)
conc = df_main.merge(df_to_group, on='UT')
conc

,CN,TC,AB,AU,C1,WC,PT,DE,SO,PY,...,EU,EM,UT,IOT,AI,computing_infrastructure,additive_manufacturing,big_data,robotics,blockchain
0,"United Kingdom, United Kingdom, United Kingdom",26,This paper presents a method for detection and...,"Ondra, V; Sever, IA; Schwingshackl, CW","[Ondra, V.; Schwingshackl, C. W.] Imperial Col...","Engineering, Mechanical",J,Non-linear system characterisation; Hilbert tr...,MECHANICAL SYSTEMS AND SIGNAL PROCESSING,2017,...,True,v.ondra14@imperial.ac.uk; Ibrahim.Sever@Rolls-...,WOS:000385209000012,False,True,False,False,False,False,False
1,"China, China, China",55,To find a better way to screen early lung canc...,"Liu, K; Kang, G","[Liu, Kui; Kang, Guixia] Beijing Univ Posts & ...","Engineering, Electrical & Electronic; Optics; ...",J,lung nodule classification; multiview convolut...,INTERNATIONAL JOURNAL OF IMAGING SYSTEMS AND T...,2017,...,False,liukui_006@126.com,WOS:000397734700002,False,True,False,False,False,False,False
2,"India, India, China",39,"Among the various fuzzy models, the well-known...","Ali, MS; Balasubramaniam, P; Zhu, QX","[Ali, M. Syed] Thiruvalluvar Univ, Dept Math, ...","Computer Science, Artificial Intelligence",J,Discrete and distributed time-varying delay; G...,INTERNATIONAL JOURNAL OF MACHINE LEARNING AND ...,2017,...,False,syedgru@gmail.com; zqx22@126.com,WOS:000393309600022,False,True,False,False,False,False,False
3,,48,"Four common food colorants, containing tartraz...","Benvidi, A; Abbasi, S; Gharaghani, S; Tezerjan...","[Benvidi, Ali; Abbasi, Saleheh; Tezerjani, Mar...","Chemistry, Applied; Food Science & Technology;...",J,Multicomponent analysis; Artificial neural net...,FOOD CHEMISTRY,2017,...,False,abenvidi@yazd.ac.ir,WOS:000388112500048,False,True,False,False,False,False,False
4,"United States, United States, United States, I...",12,"In this paper, an offline approximate dynamic ...","Rajagopal, K; Balakrishnan, SN; Busemeyer, JR","[Rajagopal, Karthikeyan] Gen Motors Co, Global...","Computer Science, Artificial Intelligence; Com...",J,Adaptive dynamic programming; neural network-b...,IEEE TRANSACTIONS ON NEURAL NETWORKS AND LEARN...,2017,...,False,krg2d@mst.edu; bala@mst.edu; jbusemey@indiana.edu,WOS:000395980500005,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1300509,"United States, United States, United States, U...",0,We report on efforts toward a monolithic Ka-ba...,"Joye, CD; Cook, AM; Jaynes, RL; Wood, FN; Albr...","[Joye, Colin D.; Cook, Alan M.; Jaynes, Regina...","Engineering, Electrical & Electronic; Physics,...",C,electron tubes; traveling wave tubes; 3D print...,IVEC 2021: 2021 22ND INTERNATIONAL VACUUM ELEC...,2021,...,False,NaN,WOS:000803073000099,False,False,False,True,False,False,False
1300510,"United States, United States, United States, U...",1,Additive manufacturing (AM) or 3D printing is ...,"Yang, W; Chen, JL; Zhang, C; Paynabar, K","[Yang, Wei; Chen, Jialei; Zhang, Chuck; Paynab...","Engineering, Multidisciplinary; Engineering, I...",J,control chart; cyber-attack detection; cyberse...,QUALITY AND RELIABILITY ENGINEERING INTERNATIONAL,2022,...,False,kamran.paynabar@isye.gatech.edu,WOS:000674032200001,False,False,False,True,False,False,False
1300511,"China, China, China, China, China",4,The adaptive processing parameters are the imp...,"Zhang, HP; Wang, JH; Liu, YL; Zhang, XS; Zhao, ZY","[Zhang, Hongping; Liu, Yaling; Zhang, Xiaoshua...","Materials Science, Multidisciplinary; Physics,...",J,3D printing; Composite cement materials; Proce...,MATERIALS LETTERS,2022,...,False,jilin44@163.com,WOS:000729362900005,False,False,False,True,False,False,False
1300512,"United States, United States, United States, U...",1,"Additive manufacturing (AM), as a means for ra...","Parker, H; Psulkowski, S; Tran, P; Dickens, T","[Parker, Helen; Psulkowski, Sean; Phong Tran; ...","Engineering, Manufacturing",C,Additive Manufacturing; 3D Printing; In-Situ A...,49TH SME NORTH AMERICAN MANU

In [41]:
df_dt_updated = conc.loc[conc['PY'] < 2022, :]
df_dt_updated.to_pickle("../data/dataframes/dt_updated.pkl")

In [40]:
df_dt

,PT,AU,TI,SO,DE,AB,C1,EM,TC,PY,...,UT,CN,AI,big_data,IOT,computing_infrastructure,blockchain,robotics,additive_manufacturing,EU
0,J,"Ondra, V; Sever, IA; Schwingshackl, CW",A method for detection and characterisation of...,MECHANICAL SYSTEMS AND SIGNAL PROCESSING,Non-linear system characterisation; Hilbert tr...,This paper presents a method for detection and...,"[Ondra, V.; Schwingshackl, C. W.] Imperial Col...",v.ondra14@imperial.ac.uk; Ibrahim.Sever@Rolls-...,26,2017,...,WOS:000385209000012,"United Kingdom, United Kingdom, United Kingdom",True,False,False,False,False,False,False,True
1,J,"Liu, K; Kang, G",Multiview convolutional neural networks for lu...,INTERNATIONAL JOURNAL OF IMAGING SYSTEMS AND T...,lung nodule classification; multiview convolut...,To find a better way to screen early lung canc...,"[Liu, Kui; Kang, Guixia] Beijing Univ Posts & ...",liukui_006@126.com,55,2017,...,WOS:000397734700002,"China, China, China",True,False,False,False,False,False,False,False
2,J,"Ali, MS; Balasubramaniam, P; Zhu, QX",Stability of stochastic fuzzy BAM neural netwo...,INTERNATIONAL JOURNAL OF MACHINE LEARNING AND ...,Discrete and distributed time-varying delay; G...,"Among the various fuzzy models, the well-known...","[Ali, M. Syed] Thiruvalluvar Univ, Dept Math, ...",syedgru@gmail.com; zqx22@126.com,39,2017,...,WOS:000393309600022,"India, India, China",True,False,False,False,False,False,False,False
3,J,"Benvidi, A; Abbasi, S; Gharaghani, S; Tezerjan...",Spectrophotometric determination of synthetic ...,FOOD CHEMISTRY,Multicomponent analysis; Artificial neural net...,"Four common food colorants, containing tartraz...","[Benvidi, Ali; Abbasi, Saleheh; Tezerjani, Mar...",abenvidi@yazd.ac.ir,48,2017,...,WOS:000388112500048,,True,False,False,False,False,False,False,False
4,J,"Rajagopal, K; Balakrishnan, SN; Busemeyer, JR",Neural Network-Based Solutions for Stochastic ...,IEEE TRANSACTIONS ON NEURAL NETWORKS AND LEARN...,Adaptive dynamic programming; neural network-b...,"In this paper, an offline approximate dynamic ...","[Rajagopal, Karthikeyan] Gen Motors Co, Global...",krg2d@mst.edu; bala@mst.edu; jbusemey@indiana.edu,12,2017,...,WOS:000395980500005,"United States, United States, United States, I...",True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253123,J,"Wei, FF; Feng, JQ",Real-time ray casting of algebraic B-spline su...,COMPUTERS & GRAPHICS-UK,Piecewise algebraic surface; B-spline; Newton-...,Piecewise algebraic B-spline surfaces (ABS sur...,"[Wei, Feifei; Feng, Jieqing] Zhejiang Univ, St...",weifeifeicad@gmail.com; jqfeng@cad.zju.edu.cn,2,2011,...,WOS:000294034600008,"China, China",False,False,False,True,False,False,False,False
1253124,C,"Hu, Y; Huang, XM; Baker, AH; Tseng, YH; Bryan,...",Improving the Scalability of the Ocean Barotro...,PROCEEDINGS OF SC15: THE INTERNATIONAL CONFERE...,parallel computing; linear solver; ocean modeling,High-resolution climate simulations are increa...,"[Hu, Yong; Huang, Xiaomeng; Yang, Guangwen] Ts...",huyong11@tsinghua.edu.cn; hxm@tsinghua.edu.cn;...,10,2015,...,WOS:000382162500043,"China, China, China, United States, United Sta...",False,False,False,True,False,False,False,False
1253125,J,"Kim, TJ; Sun, X; Yoon, SE",T-ReX: Interactive Global Illumination of Mass...,IEEE TRANSACTIONS ON VISUALIZATION AND COMPUTE...,Massive models; ray tracing; photon mapping; g...,We propose several interactive global illumina...,"[Kim, Tae-Joon; Yoon, Sung-Eui] Korea Adv Inst...",tjkim.kaist@gmail.com; sunxin@microsoft.com; s...,6,2014,...,WOS:000338118700013,"South Korea, South Korea, China",False,False,False,True,False,False,False,False
1253126,J,"Kavouklis, C; Kallinderis, Y",Parallel adaptation of general three-dimension...,JOURNAL OF COMPUTATIONAL PHYSICS,Parallel computing; Hybrid meshes; Mesh adapta...,A new parallel dynamic mesh adaptation and loa...,"[Kavouklis, Christos; Kallinderis, Yannis] Uni...",christos.kavou